# Sea ice and meridional heat transport climatology

In [1]:
import intake
import matplotlib.pyplot as plt
import numpy as np
import netCDF4 as nc
import cartopy.crs as ccrs
import xarray as xr
import cmocean as cm
import glob
import matplotlib.colors as col
import cf_xarray as cf
# need to install opencv-python for this:
import cv2
import warnings
import logging
from scipy.ndimage import distance_transform_edt
import scipy.ndimage as nd
from scipy.interpolate import griddata
from matplotlib.animation import FuncAnimation
import imageio.v3 as iio
import glob
import imageio.v3 as iio
import cmocean.cm as cm

logging.captureWarnings(True)
logging.getLogger('py.warnings').setLevel(logging.ERROR)

from dask.distributed import Client


In [2]:
client = Client(threads_per_worker = 1)

/g/data/xp65/public/apps/med_conda/envs/analysis3-25.10/lib/python3.11/site-packages/distributed/diagnostics/nvml.py:14: FutureWarning: The pynvml package is deprecated. Please install nvidia-ml-py instead. If you did not install pynvml directly, please report this to the maintainers of the package that installed pynvml for you.
  import pynvml
/g/data/xp65/public/apps/med_conda/envs/analysis3-25.10/lib/python3.11/site-packages/distributed/diagnostics/nvml.py:14: FutureWarning: The pynvml package is deprecated. Please install nvidia-ml-py instead. If you did not install pynvml directly, please report this to the maintainers of the package that installed pynvml for you.
  import pynvml
/g/data/xp65/public/apps/med_conda/envs/analysis3-25.10/lib/python3.11/site-packages/distributed/diagnostics/nvml.py:14: FutureWarning: The pynvml package is deprecated. Please install nvidia-ml-py instead. If you did not install pynvml directly, please report this to the maintainers of the package that i

In [3]:
#in this definition, no frequency can be used only for static data
iaf_run = '01deg_jra55v140_iaf_cycle3'
catalog = intake.cat.access_nri
#defining importing function
def importer(experiment,catalog,variable, frequency = 'fx',start_time=0,end_time=0):
    warnings.filterwarnings('ignore')
    var = catalog[experiment].search(variable=variable, frequency = frequency).to_dask(xarray_open_kwargs={'decode_timedelta':True})
    if start_time!=0:
        time_slice='slice(start_time,end_time)'
        return eval("var." + variable + ".sel(time=" + time_slice + ")")
    elif start_time ==0:
        return eval("var." + variable)

# defining list variables function
def intakelist(catalog,experiment):
    return catalog[experiment].unique().variable

## I - Calculating climatologies

Climatologies will be calculated once and saved into a netcdf file, which means we only need to run ths section *I* of the code, **once**

Calculating sea ice extent climatology

In [4]:
#### This cells takes a HUGE TIME to run
#importing area
area_icemodel = importer(iaf_run,catalog,variable='tarea', frequency = '1mon').isel(nj=slice(1,800)).compute()
#importing sea ice fraction
SIF = importer(iaf_run,catalog,variable='aice_m', frequency = '1mon').isel(nj=slice(1,800))
#setting SIF limit for SIE
SIF_limit = 0.15 #limit of fraction below which we consider the cell ice free for ice extent calculations
#filtering out ice-free cells in SIF
SIF_forextent = SIF.where(SIF >SIF_limit)
#calcualting sea ice extent
SIE = (SIF_forextent * area_icemodel).sum(('nj','ni'))
#calculating monthly climatology
SIE_clim = SIE.groupby('time.month').mean().compute()

calculating sea ice area

In [5]:
# monthly sea ice area
SIA = (SIF * area_icemodel).sum(('nj','ni'))
#calculating monthly climatology
SIA_clim = SIA.groupby('time.month').mean().compute()

calculating climatology of zonal means of meridional heat transport

In [6]:
lat_slice=slice(-90,-45)
#importing sea ice fraction
Ty = importer(iaf_run,catalog,variable='temp_yflux_adv', frequency = '1mon').sel(yu_ocean=lat_slice)
#calculating zonal sum
Ty_ZM =Ty.sum('xt_ocean')
#calculating climatology of full data and zonal means
Tyfull_clim = Ty.groupby('time.month').mean().compute()
TyZM_clim = Ty_ZM.groupby('time.month').mean().compute()

In [ ]:
# saving sea ice values and variables
savedir = '/home/156/wf4500/Tiago_Dotto_SIC_Eddies/OM2_Eddies_SeaIce/Plotted_data/'

#sea ice extent and area
SIE_clim.name = 'SIE_m2'
SIA_clim.name = 'SIA_m2'
SIE_clim.to_netcdf(savedir+'SIE_climatology_OM2_IAF.nc')
SIA_clim.to_netcdf(savedir+'SIA_climatology_OM2_IAF.nc')

# heat transports
TyZM_clim.name = 'MHT_xsum_watts'
Tyfull_clim.name = 'MHT_watts'
Tyfull_clim.to_netcdf(savedir+'MHT_climatology_OM2_IAF.nc')
TyZM_clim.to_netcdf(savedir+'MHT_ZM_climatology_OM2_IAF.nc')

There are 1 HDF5 objects open!

Report: open objects on 72057594037927938


[gadi-hmem-clx-0012:3710068:0:3710068] Caught signal 11 (Segmentation fault: address not mapped to object at address 0x5b0)
BFD: Dwarf Error: Can't find .debug_ranges section.
BFD: Dwarf Error: Can't find .debug_ranges section.
BFD: Dwarf Error: Can't find .debug_ranges section.
BFD: Dwarf Error: Can't find .debug_ranges section.
BFD: Dwarf Error: Can't find .debug_ranges section.
BFD: Dwarf Error: Can't find .debug_ranges section.
BFD: Dwarf Error: Can't find .debug_ranges section.
BFD: Dwarf Error: Can't find .debug_ranges section.
BFD: Dwarf Error: Can't find .debug_ranges section.
BFD: Dwarf Error: Can't find .debug_ranges section.
BFD: Dwarf Error: Can't find .debug_ranges section.
BFD: Dwarf Error: Can't find .debug_ranges section.
BFD: Dwarf Error: Can't find .debug_ranges section.
BFD: Dwarf Error: Can't find .debug_ranges section.
BFD: Dwarf Error: Can't find .debug_ranges section.
BFD: Dwarf Error: Can't find .debug_ranges section.
BFD: Dwarf Error: Can't find .debug_ranges s

## II - Plotting climatology of heat transports and sea ice

Importing climatoloties again

In [ ]:
savedir = '/home/156/wf4500/Tiago_Dotto_SIC_Eddies/OM2_Eddies_SeaIce/Plotted_data/'

#sea ice
SIE_clim = xr.open_dataset(savedir+'SIE_climatology_OM2_IAF.nc')
SIA_clim = xr.open_dataset(savedir+'SIA_climatology_OM2_IAF.nc')

# heat transports

Tyfull_clim = xr.open_dataset(savedir+'MHT_climatology_OM2_IAF.nc')
TyZM_clim = xr.open_dataset(savedir+'MHT_ZM_climatology_OM2_IAF.nc')